# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [28]:
import sqlite3
sqlite_db = '../../assets/datasets/enron.db'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [29]:
from pandas.io import sql
import pandas as pd

In [30]:
pwd

u'/vagrant/DSI_JY/DSI-projects/classes/week-05/1.3-lab/code/starter-code'

In [31]:
results = c.execute('select * from messagebase limit 10')
results.fetchall()

[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138),
 (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138),
 (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138),
 (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138),
 (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138),
 (6, u'taylor-m/sent/31', 912570420, u'Re: Position Description', 138),
 (7, u'taylor-m/sent/33', 912576240, u'Koch', 138),
 (8, u'taylor-m/sent/40', 912685080, u'Re: Time to Celebrate!', 138),
 (9, u'taylor-m/sent/41', 912734100, u'Re: Vacation Request', 138),
 (10, u'taylor-m/sent/44', 913166040, u'Re: Last Message', 138)]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [32]:
results = c.execute("""SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;""")
results.fetchall()

[(u'EmployeeBase',), (u'MessageBase',), (u'RecipientBase',)]

In [33]:
results = c.execute("""PRAGMA table_info(EmployeeBase);""")
results.fetchall()

[(0, u'eid', u'INTEGER', 0, None, 0),
 (1, u'name', u'TEXT', 0, None, 0),
 (2, u'department', u'TEXT', 0, None, 0),
 (3, u'longdepartment', u'TEXT', 0, None, 0),
 (4, u'title', u'TEXT', 0, None, 0),
 (5, u'gender', u'TEXT', 0, None, 0),
 (6, u'seniority', u'TEXT', 0, None, 0)]

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [34]:
results = c.execute("""select * from EmployeeBase limit 5;""")
results.fetchall()

[(1,
  u'John Arnold',
  u'Forestry',
  u'ENA Gas Financial',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (2,
  u'Harry Arora',
  u'Forestry',
  u'ENA East Power',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (3,
  u'Robert Badeer',
  u'Forestry',
  u'ENA West Power',
  u'Mgr Trading',
  u'Male',
  u'Junior'),
 (4,
  u'Susan Bailey',
  u'Legal',
  u'ENA Legal',
  u'Specialist Legal',
  u'Female',
  u'Junior'),
 (5,
  u'Eric Bass',
  u'Forestry',
  u'ENA Gas Texas',
  u'Trader',
  u'Male',
  u'Junior')]

In [35]:
results = c.execute("""select * from MessageBase limit 5;""")
results.fetchall()

[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138),
 (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138),
 (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138),
 (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138),
 (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)]

In [36]:
results = c.execute("""select * from RecipientBase limit 5;""")
results.fetchall()

[(1, 1, 59), (2, 1, 15), (3, 1, 15), (4, 1, 109), (4, 2, 49)]

Import each of the 3 tables to a Pandas Dataframes

In [37]:
df_employee = sql.read_sql('''select * from EmployeeBase ''', con=conn)

In [38]:
df_msg = sql.read_sql('''select * from MessageBase ''', con=conn)

In [39]:
df_recipient = sql.read_sql('''select * from RecipientBase ''', con=conn)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [40]:
len(df_employee)

156

In [41]:
len(df_msg)

21635

In [47]:
df_msg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21635 entries, 0 to 21634
Data columns (total 5 columns):
mid          21635 non-null int64
filename     21635 non-null object
unix_time    21635 non-null int64
subject      21635 non-null object
from_eid     21635 non-null int64
dtypes: int64(3), object(2)
memory usage: 845.2+ KB


In [27]:
import datetime

In [44]:
datetime.datetime.fromtimestamp(910930020)

datetime.datetime(1998, 11, 13, 12, 7)

In [50]:
df_msg['unix_time'] =  pd.to_datetime(df_msg['unix_time'],unit='s')

In [51]:
df_msg['unix_time'].max()

Timestamp('2002-06-21 13:37:34')

In [52]:
df_msg['unix_time'].min()

Timestamp('1998-11-13 04:07:00')

In [53]:
df_msg.head()

mid          filename           unix_time                    subject  \
0    1  taylor-m/sent/11 1998-11-13 04:07:00             Cd$ CME letter   
1    2  taylor-m/sent/17 1998-11-19 07:19:00            Indemnification   
2    3  taylor-m/sent/18 1998-11-19 08:24:00        Re: Indemnification   
3    4  taylor-m/sent/23 1998-11-24 02:23:00     Re: Coral Energy, L.P.   
4    5  taylor-m/sent/27 1998-11-30 03:22:00  Bankruptcy Code revisions   

   from_eid  
0       138  
1       138  
2       138  
3       138  
4       138

In [55]:
len(df_msg['mid'].unique())

21635

In [56]:
df_recipient.head()

mid  rno  to_eid
0    1    1      59
1    2    1      15
2    3    1      15
3    4    1     109
4    4    2      49

In [57]:
df_recipient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38388 entries, 0 to 38387
Data columns (total 3 columns):
mid       38388 non-null int64
rno       38388 non-null int64
to_eid    38388 non-null int64
dtypes: int64(3)
memory usage: 899.8 KB


In [59]:
df_count = df_recipient.groupby('mid')[['rno']].count()

In [63]:
(df_count['rno']==1).sum()

14985

In [64]:
(df_count['rno']==2).sum()

2962

In [66]:
(df_count['rno']>=5).sum()

1380

In [69]:
df_count[df_count['rno']==57]

rno
mid       
12116   57
12151   57

Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.